# Research Track 2  --- Assignment 2 ---

In [1]:
#!/usr/bin/env python3
%matplotlib widget

import sys
sys.path.append('/root/ros_ws/devel/lib/python3/dist-packages')
import rospy
import threading
import actionlib
import sensor_msgs.msg
import ipywidgets as widgets
import matplotlib.pyplot as plt
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Twist
from my_assignment2.msg import PositionVelocity
from assignment_2_2024.msg import PlanningAction, PlanningGoal
from IPython.display import display, clear_output
from matplotlib.animation import FuncAnimation
from ipywidgets import HBox

In [2]:
# --- Global publishers ---
robot_state_pub = None
cmd_vel_pub = None

# --- Global variable ---
robot_state = PositionVelocity()
goal_client = None
cancel_requested = False

# Lists of past x and y positions of robot
positions_x = []
positions_y = []

# Counters for reached and not reached goals
num_reached = 0
num_not_reached = 0

## 1. Interface to assign or cancel goals

In [3]:
# Input
x_input = widgets.FloatText(
    description='Target x:',
    value=1.0
)
y_input = widgets.FloatText(
    description='Target y:',
    value=1.0
)

# Send and cancel goal
send_goal_button = widgets.Button(
    description='Send Goal',
    button_style='success'
)

cancel_goal_button = widgets.Button(
    description='Cancel Goal',
    button_style='danger'
)

display(x_input, y_input)
button_box = HBox([send_goal_button, cancel_goal_button])
display(button_box)

FloatText(value=1.0, description='Target x:')

FloatText(value=1.0, description='Target y:')

[INFO] [1746805657.866271, 2446.388000]: Goal sending process...
[INFO] [1746805657.874207, 2446.399000]: Goal sent: x=3.0, y=1.0
[INFO] [1746805672.857119, 2460.719000]: Goal sending process...
[INFO] [1746805672.864077, 2460.723000]: Goal sent: x=3.0, y=2.0


## 2. Trajectory and positions of the robot

In [4]:
# Label to display current robot position
robot_position_label = widgets.HTML(
    value='<b>Current Robot Position<b>: (x=0.0, y=0.0)'
)
display(robot_position_label)

out1 = widgets.Output()
plt.ioff()  # auto-display無効化
fig, ax = plt.subplots()
ax.set_xlim(-10, 10)
ax.set_ylim(-10, 10)
ax.set_title('Robot Trajectory')
line, = ax.plot([], [], 'b-')

with out1:
    display(fig.canvas)  # ✅ display(fig.canvas) → canvasオブジェクトを直接渡す

def init():
    return line,

def update(frame):
    global positions_x, positions_y
    line.set_data(positions_x, positions_y)
    return line,

ani = FuncAnimation(fig, update, init_func=init, blit=False, interval=500)

display(out1)

HTML(value='<b>Current Robot Position<b>: (x=0.0, y=0.0)')

Output()

## 3. Distance from the closest obstacle

In [5]:
obstacle_distance_label = widgets.HTML(
    value='<b>Closest Obstacle Distance<b>: Unknown'
)
display(obstacle_distance_label)

HTML(value='<b>Closest Obstacle Distance<b>: Unknown')

## 4. The number of reached/not-reached targets

In [6]:
out2 = widgets.Output()
global num_reached, num_not_reached

with out2:
    plt.ioff()
    fig, ax = plt.subplots()
    bars = ax.bar(['Reached', 'Not Reached'], [num_reached, num_not_reached], color=['green', 'red'])
    ax.set_ylim(0, 10)
    ax.set_title('The number of reached/not-reached targets')
    fig.tight_layout()
    fig.canvas.header_visible = False

    display(fig.canvas)
    
def update_bar():
    bars[0].set_height(num_reached)
    bars[1].set_height(num_not_reached)
    fig.canvas.draw_idle()  
    
display(out2)

Output()

### Functions

In [7]:
def odom_callback(msg):
    # Callback function for the /odom topic
    global robot_state
    
    robot_state.x = msg.pose.pose.position.x
    robot_state.y = msg.pose.pose.position.y
    robot_state.vel_x = msg.twist.twist.linear.x
    robot_state.vel_z = msg.twist.twist.angular.z
    
    # Store the position
    positions_x.append(robot_state.x)
    positions_y.append(robot_state.y)
    
    # Update the robot current location
    robot_position_label.value = f"<b>Current Robot Position<b>: (x={robot_state.x:.2f}, y={robot_state.y:.2f})"

    # Publish the robot status
    robot_state_pub.publish(robot_state)

In [8]:
def scan_callback(msg):
    # Callback function for the /scan topic
    valid_ranges = [r for r in msg.ranges if not (r == float('inf') or r != r)]
    if valid_ranges:
        closest_obstacle = min(valid_ranges)
    else:
        closest_obstacle = float('inf')
    obstacle_distance_label.value = f"<b>Closest Obstacle Distance:<b> {closest_obstacle:.2f} m"

In [9]:
def check_goal_result_callback(status, result):
    # Callback function to check the goal status
    global num_reached, num_not_reached
    
    print("check_goal_result_callback is called")
    
    if status == actionlib.GoalStatus.SUCCEEDED:
        print("The goal has been reached!")
        num_reached += 1
        rospy.loginfo(num_reached)
        rospy.loginfo(num_not_reached)

    elif status in [actionlib.GoalStatus.PREEMPTED, actionlib.GoalStatus.ABORTED]:
        rospy.loginfo("The goal has not been reached!")
        num_not_reached += 1
    else:
        rospy.loginfo(f"goal status: {status}")
    update_bar()

In [10]:
def send_goal_handler(b):
    # Function to send the goal when the 'send_goal_button' is pressed 
    global cancel_requested
    rospy.loginfo("Goal sending process...")

    rospy.set_param('/des_pos_x', x_input.value)
    rospy.set_param('/des_pos_y', y_input.value)

    goal = PlanningGoal()
    goal.target_pose.header.frame_id = "map" 
    goal.target_pose.pose.position.x = x_input.value
    goal.target_pose.pose.position.y = y_input.value
    goal.target_pose.pose.orientation.w = 1.0 

    goal_client.send_goal(goal, done_cb=check_goal_result_callback)
    cancel_requested = False
    rospy.loginfo(f"Goal sent: x={x_input.value}, y={y_input.value}")

In [11]:
def cancel_goal_handler(b):
    # Function to send the goal when the 'cancel_goal_button' is pressed
    global cancel_requested, num_not_reached

    if goal_client.get_state() in [actionlib.GoalStatus.ACTIVE, actionlib.GoalStatus.PENDING]:
        print(goal_client.get_state())
        goal_client.cancel_goal()
        stop_robot()
        cancel_requested = True
        rospy.logwarn("Goal canceled by user.")
    else:
        rospy.logwarn("No active goal to cancel.")

In [12]:
def stop_robot():
    # Function to stop the robot
    stop_msg = Twist()
    stop_msg.linear.x = 0.0
    stop_msg.angular.z = 0.0
    cmd_vel_pub.publish(stop_msg)
    rospy.loginfo("Robot stopped.")

### Main code

In [13]:
# Register a event for a button
send_goal_button.on_click(send_goal_handler)
cancel_goal_button.on_click(cancel_goal_handler)

rospy.init_node('jupyter_action_client', anonymous=False)

# Initialize publishers and subscriber
robot_state_pub = rospy.Publisher('/robot_state', PositionVelocity, queue_size=10)
cmd_vel_pub = rospy.Publisher('/cmd_vel', Twist, queue_size=10)
odom_sub = rospy.Subscriber('/odom', Odometry, odom_callback)
scan_sub = rospy.Subscriber('/scan', sensor_msgs.msg.LaserScan, scan_callback)

# Create the action client
goal_client = actionlib.SimpleActionClient('/reaching_goal', PlanningAction)
rospy.loginfo("Waiting for action server...")
goal_client.wait_for_server()
rospy.loginfo("Action server ready.")

[INFO] [1746805650.442722, 2439.342000]: Waiting for action server...
[INFO] [1746805650.490433, 2439.388000]: Action server ready.
